In [33]:
def read_data(filename):
    """
    Считывает данные из файла, формируя двумерный список.

    Args:
        filename: Имя файла с данными.

    Returns:
        Двумерный список, где каждый подсписок содержит 10 значений.
        Возвращает None, если файл не может быть открыт или формат файла неверен.
    """
    data = []
    current_list = []
    try:
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()  # Удаляем пробелы в начале и конце строки
                if not line:  # Пропускаем пустые строки
                    continue

                try:
                    index, value = line.split('\t')  # Разделяем строку на индекс и значение (разделитель - табуляция)
                    value = float(value)  # Преобразуем значение в число с плавающей точкой
                    current_list.append(value)
                except ValueError:
                    print(f"Ошибка: Неверный формат строки: {line}")
                    return None  # Прерываем чтение файла

                if len(current_list) == 10:
                    data.append(current_list)
                    current_list = []  # Начинаем новый подсписок

        # Проверяем, что количество элементов в последнем списке равно 10 (если есть неполный список)
        if current_list:
            print("Предупреждение: Последний список неполный.")
            #Можно добавить обработку неполного списка, например, дополнить нулями
            data.append(current_list + [0] * (10 - len(current_list)))
            # Или не добавлять его вовсе
            #pass


        return data

    except FileNotFoundError:
        print(f"Ошибка: Файл не найден: {filename}")
        return None
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return None

In [34]:
def print_mass(mass):
    for row in mass:
        for elem in row:
            print(f"{elem:6.2f}", end = '')
        print()
    

In [42]:
import numpy as np
import scipy.stats as stats

def weibull_binning(data, num_bins=6): # Добавлен plot_fit
    # 1. Преобразование данных в одномерный массив
    flat_data = data.flatten()

    # 2. Сдвиг данных (если необходимо)
    min_val = np.min(flat_data)
    if min_val < 0:
        shift = -min_val
        flat_data = flat_data + shift
    else:
        shift = 0

    # 3. Оценка параметров распределения Вейбулла
    try:
        k, loc, scale = stats.weibull_min.fit(flat_data, floc=0)
    except RuntimeError:
        print("Не удалось оценить параметры распределения Вейбулла.")
        return None, None
    except Exception as e: #Добавлено для перехвата других возможных исключений
        print(f"Произошла ошибка при оценке параметров Вейбулла: {e}")
        return None, None

    # 4. Создание границ отрезков
    percentiles = np.linspace(0, 100, num_bins + 1)
    bin_edges = stats.weibull_min.ppf(percentiles / 100, k, loc, scale)

    # 5. Сдвиг границ обратно
    bin_edges = bin_edges - shift

    # 6. Применение биннинга
    binned_flat_data = np.digitize(flat_data, bin_edges, right=False) - 1

    # 7. Обработка значений, выходящих за границы
    binned_flat_data[binned_flat_data == -1] = 0
    binned_flat_data[binned_flat_data == num_bins] = num_bins - 1

    # 8. Возврат к исходной форме
    binned_data = binned_flat_data.reshape(data.shape)

    # 9. Визуализация (по желанию) - для диагностики
    bin_edges[-1] = max(data).item() +0.001
    return bin_edges

In [36]:
data_temp_all = read_data("decANDsrTutf8.txt")

print_mass(data_temp_all[:-1])


Предупреждение: Последний список неполный.
 -9.19 -9.60-15.28-13.67-14.23-10.65 -1.22 -4.23 -2.42  0.43
-23.23-11.55 -9.31-15.05 -8.09-12.80  0.27 -0.01 -0.36  3.96
-20.21-16.05-14.86 -2.89  0.64 -0.50 -3.57  0.40  1.36  1.13
 -4.20 -6.30 -9.27-13.53 -6.63 -5.22 -0.84 -3.33 -0.60  2.74
-14.67 -8.91 -9.05 -5.88 -5.18 -5.85 -7.81 -2.51 -1.50  2.78
-11.90-17.46-17.21 -6.70-12.32 -6.92  1.85  0.32  1.00  4.15
-17.67-20.19-15.21 -3.59 -5.42 -7.14  0.60 -0.04  1.10  4.20
-20.59-16.25 -7.40 -8.07 -7.66-12.30 -7.36  0.26  2.71  1.77
 -3.94 -5.96 -6.76-16.32-17.19-15.93 -6.47  1.70  1.45  3.79
 -7.59 -4.45 -5.92 -1.26 -5.43 -3.41 -6.34  2.16  0.27 -0.90
-11.16 -5.23  0.95  0.32  0.01 -4.36  0.23 -0.30  0.22  5.00
 -3.06 -2.89 -4.59 -3.91 -3.82 -2.23  0.55 -0.02  3.72  6.77
 -8.89 -4.56 -9.31 -4.02 -1.15 -2.66 -3.33 -4.82 -1.03  0.28
-12.21 -9.24-16.43-14.74-11.55-14.53 -1.80 -1.44 -0.32  2.52
-10.90 -2.88 -5.49 -9.20 -3.53 -0.81 -1.50 -0.63  1.29  3.61
-11.01  0.81 -3.11 -3.58 -1.66 -3.36 -2.36

In [43]:
data_year = np.array(data_temp_all[:-1])
data_temp_pi = data_temp_all[-1][0]

temp_diapazon = [weibull_binning(data_year[:, i:i+1])[1:] for i in range(10)]
print_mass(temp_diapazon)


def mass_diap_temp(diap, point):
    #В каком диапазоне находится данная температура
    Temp_in_diap = []
    for year in data_year:
        i = 0
        while year[point] >= diap[i]:
            i+=1
        Temp_in_diap.append(i)
    return Temp_in_diap


def count_pereh(len_temp_d1, len_temp_d2, T_in_diap1, T_in_diap2):
    Diap = [[0 for k in range(len_temp_d2)]for i in range(len_temp_d1)]
    for i in range(len(T_in_diap1)):
        Diap[T_in_diap1[i]][T_in_diap2[i]] += 1
    return Diap

def form_P_m(dp):
    p_m = [[0 for i in range(len(dp[0]))] for i in range((len(dp)))]
    for i in range(len(dp)):
        for j in range((len(dp[0]))):
            if sum(dp[i]) == 0:
                p_m[i][j]
            else:
                p_m[i][j] = dp[i][j] / sum(dp[i])
    print_mass(p_m)
    return p_m

def transpon_matr(mass): 
    
    rows = len(mass)
    cols = len(mass[0])

    transposed_matrix = [[0 for _ in range(rows)] for _ in range(cols)]

    # Заполняем транспонированную матрицу
    for i in range(rows):
        for j in range(cols):
            transposed_matrix[j][i] = mass[i][j]

    return transposed_matrix

Temp_in_diap = []
Temp_in_diap.append(mass_diap_temp(temp_diapazon[0], 0))

Pi_10 = [0 for i in temp_diapazon[0]]
j = 0
while data_temp_pi >= temp_diapazon[0][j]:
    j+=1
Pi_10[j] = 1

for i in range(9):
    Temp_in_diap.append(mass_diap_temp(temp_diapazon[i+1], i+1))
    diap = count_pereh(len(temp_diapazon[i]), len(temp_diapazon[i+1]), Temp_in_diap[i], Temp_in_diap[i+1])
    P_m = form_P_m(diap)
    P_m_t = transpon_matr(P_m)
    try:
        Pi_10 = np.dot(np.array(P_m_t), np.array(Pi_10))
        print(Pi_10)
    except np.linalg.LinAlgError:
        print("Матрица A сингулярна (нет решения или бесконечно много решений)")


    

-15.89-13.39-11.24 -9.03 -6.24 -3.06
-14.61-11.69 -9.19 -6.62 -3.37  0.81
-14.46-12.35-10.17 -7.57 -3.79  0.95
-12.44-10.33 -8.31 -6.02 -2.89  0.32
 -9.42 -7.31 -5.58 -3.86 -1.74  0.64
 -9.76 -7.74 -6.03 -4.27 -2.06 -0.50
 -5.15 -3.89 -2.74 -1.47  0.21  1.85
 -2.34 -1.45 -0.68  0.12  1.15  2.28
 -0.78 -0.13  0.44  1.05  1.84  3.72
  0.99  1.93  2.72  3.51  4.50  6.77
  0.75  0.00  0.25  0.00  0.00  0.00
  0.50  0.00  0.00  0.50  0.00  0.00
  0.50  0.00  0.50  0.00  0.00  0.00
  0.17  0.00  0.17  0.00  0.17  0.50
  0.00  0.00  0.00  0.00  1.00  0.00
  0.00  0.00  0.00  0.25  0.50  0.25
[0. 0. 0. 0. 1. 0.]
  0.67  0.00  0.00  0.00  0.17  0.17
  0.00  0.00  0.00  0.00  0.00  0.00
  0.67  0.00  0.00  0.33  0.00  0.00
  0.00  0.00  0.50  0.50  0.00  0.00
  0.00  0.00  0.00  0.40  0.40  0.20
  0.00  0.00  0.00  0.00  0.75  0.25
[0.  0.  0.  0.4 0.4 0.2]
  0.50  0.00  0.00  0.17  0.33  0.00
  0.00  0.00  0.00  0.00  0.00  0.00
  0.00  1.00  0.00  0.00  0.00  0.00
  0.50  0.00  0.00  0.00  0.5

In [44]:
print(f"[{0.1:5.3}; {temp_diapazon[9][0]:5.3}]: {(Pi_10[0]*100):5.3}%")
for i in range(1, len(temp_diapazon[9])):
    print(f"[{temp_diapazon[9][i-1]:5.3}; {temp_diapazon[9][i]:5.3}]: {(Pi_10[i]*100):5.3}%")

[  0.1; 0.995]:  19.9%
[0.995;  1.93]:  14.9%
[ 1.93;  2.72]:  10.1%
[ 2.72;  3.51]:  15.1%
[ 3.51;   4.5]:  30.1%
[  4.5;  6.77]:  9.89%


In [30]:
#Определяем матрицу Pi
Pi = [0 for i in temp_diapazon1]
i = 0
while data_temp_pi >= temp_diapazon1[i]:
    i+=1
Pi[i] = 1
print(Pi)

[0, 0, 0, 1, 0]


In [28]:
#Сколько каких переходов было
# Diap = [[0 for i in temp_diapazon1] for i in temp_diapazon2]
# for year in Temp_in_diap:
#     for i in range(len(year) - 1):
#         Diap[year[i]][year[i+1]] +=1
# print_mass(Diap)

diap = count_pereh(len(temp_diapazon1), len(temp_diapazon2), Temp_in_diap1, Temp_in_diap2)
print_mass(diap)


#Вероятности переходов = число переходов в этой ячейке / суммарное число переходов в строке
# P_m = [[0 for i in temp_diapazon] for i in temp_diapazon]
# for i in range(len(temp_diapazon)):
#     for j in range((len(temp_diapazon))):
#         P_m[i][j] = Diap[i][j] / sum(Diap[i])
# print_mass(P_m)

  0.00  0.00  0.00  1.00  0.00  0.00
  0.00  0.00  2.00  1.00  0.00  0.00
  0.00  0.00  2.00  2.00  0.00  0.00
  1.00  3.00  1.00  2.00  1.00  0.00
  0.00  0.00  0.00  3.00  0.00  1.00


In [29]:
P_m = form_P_m(diap)

  0.00  0.00  0.00  1.00  0.00  0.00
  0.00  0.00  0.67  0.33  0.00  0.00
  0.00  0.00  0.50  0.50  0.00  0.00
  0.12  0.38  0.12  0.25  0.12  0.00
  0.00  0.00  0.00  0.75  0.00  0.25


In [33]:
P_m_t = transpon_matr(P_m)
print_mass(P_m_t)

  0.00  0.00  0.00  0.12  0.00
  0.00  0.00  0.00  0.38  0.00
  0.00  0.67  0.50  0.12  0.00
  1.00  0.33  0.50  0.25  0.75
  0.00  0.00  0.00  0.12  0.00
  0.00  0.00  0.00  0.00  0.25


In [36]:

try:
    Pi_10 = np.dot(np.array(P_m_t), np.array(Pi))
    print(Pi_10)
except np.linalg.LinAlgError:
    print("Матрица A сингулярна (нет решения или бесконечно много решений)")

[0.125 0.375 0.125 0.25  0.125 0.   ]
